#### Цель этого проекта: составить модель, которая будет предсказывать рейтинг ресторана на сайте Trip Advisor. Задача  - выделить характеристики ресторана, которые можно будет использовать в построении этой модели.

In [ ]:
import pandas as pd  # импортируем все нужные библиотеки
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import numpy as np

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
DATA_DIR = '/kaggle/input/trpadvsr/'
df_train = pd.read_csv('/kaggle/input/trpdvsr/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task VS.csv')
sample_submission = pd.read_csv(DATA_DIR+'sample_submission VS.csv')

In [ ]:
df_train.head()

In [ ]:
sample_submission.info()

In [ ]:
df_train.info() #здесь мы имеем дело с базой данных Trip Advisor, содержащей 10 столбцов и 40 тыс. строк -
#данные по 40 тыс. ресторанов, разделенные на 10 параметров

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

df = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
df.info() #проверка - да все объединилось, как надо.

Что означают признаки:

City: Город,
Cuisine Style: Кухня,
Ranking: Ранг ресторана относительно других ресторанов в этом городе,
Price Range: Цены в ресторане в 3 категориях,
Number of Reviews: Количество отзывов,
Reviews: 2 последних отзыва и даты этих отзывов,
URL_TA: страница ресторана на 'www.tripadvisor.com',
ID_TA: ID ресторана в TripAdvisor,
Rating: Рейтинг ресторана

In [ ]:
df.head()

## City

In [ ]:
#проверим колонку "город" на наличие пропусков
df.City.isna().sum()

In [ ]:
#дальше категориальные признаки нужно заменить числовыми. Сначала посмотрим на список городов и проверим,
#нет ли неправильных написаний или каких-то других багов, которые могут дублировать значения
df['City'].value_counts()

In [ ]:
#Далее я попробовала сократить список значимых городов, но это никак не повлияло на модель, 
#поэтому я не вставляю этот шаг в финальное решение:
#Всего у нас 40 тыс. записей, и если следовать формуле Парето 80/20, то нас интересуют только те города, которые
#вносят вклад в 80% записей. это - 32 тыс. записей


In [ ]:
#df['City'].value_counts()[:16].sum() # это - первые 16 городов, то есть половина всей выборки

In [ ]:
#city_short = df['City'].value_counts()[:16]

In [ ]:
#city_short = city_short.reset_index()
#city_short['index'].tolist()

In [ ]:
#Оставим в базе только первые 16 городов - примерно половину, а остальные назовем other
#city_set = set()
#for item in city_short['index']:
 #   city_set.add(item)

In [ ]:
#len(city_set)

In [ ]:
#def find_item(cell):
 #   if item in cell:
  #      return 1
   # return 0
#функция, которая вернет dummies в колонки с популярными городами

In [ ]:
#for item in city_set:
 #   df[item] = df['City'].apply(find_item)

In [ ]:
#далее применим get_dummies:
df = pd.get_dummies(df, columns=['City'], dummy_na=True)

In [ ]:
display(df.head())

In [ ]:
df.info()

In [ ]:
df['City_nan'].value_counts() #поскольку распределения нет, удалим эту колонку.

In [ ]:
df = df.drop('City_nan', axis=1)

## Cuisine style

In [ ]:
#посмотрим, есть ли пропуски
df.info()

In [ ]:
df['Cuisine Style'][0] #посмотрим, что надпись в ячейке организована как список разных видов кухонь

In [ ]:
df['Cuisine Style'] = df['Cuisine Style'].fillna('X]') #заменим пустые значения на X с лишней скобкой в конце для более простой обработки в дальнейшем

In [ ]:
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: x[:-1]) #удалим лишние скобки в конце каждой строки. здесь нам и пригодилась скобка у empty

In [ ]:
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: x.split(',')) #разделим кажду строку на элементы списка

In [ ]:
df['Cuisine Style'][1]

In [ ]:
len(df['Cuisine Style'][0]) #проверим, разбились ли строки на список

In [ ]:
df['Cuisine Style'][0] #проверим, как выглядит список. Есть лишние кавычки.

In [ ]:
#напишем функцию, которая очищает все элементы всех строк от лишних кавычек и запятых
def cut_c(line):
    new_list=[]
    for element in line:
        if element=='X':
            return 'X'
        else:
            new_list.append(element[2:-1])
    return new_list

In [ ]:
df['Cuisine new'] = df['Cuisine Style'].apply(cut_c) #применим функцию

In [ ]:
df['Cuisine new'] #ура! Получилось! я избавилась от лишних скобок и всего прочего!!!

In [ ]:
cuisine_freq= pd.DataFrame(df['Cuisine new'].tolist()).stack().value_counts()
# здесь я смотрю на распределение разных видов кухонь по популярности

In [ ]:
cuisine_freq = cuisine_freq.reset_index()

In [ ]:
cuisine_freq.columns = ['index', 'freq']

In [ ]:
pd.DataFrame(df['Cuisine new'].tolist()).stack().value_counts().sum()
#посмотрим, сколько всего разных упоминаний кухонь в датасете

In [ ]:
pd.DataFrame(df['Cuisine new'].tolist()).stack().value_counts().sum() *0.8 #запуталась, как посчитать этот процентиль формулами,
#поэтому делаю вручную: 80% от всех упоминаний кухнь это - 105012.8

In [ ]:
cuisine_freq.freq[:23].sum()

In [ ]:
f_tolist = cuisine_freq['index'][0:23].tolist()

In [ ]:
#Оставим в базе только первые 23 вида кухни - они составляют 80% всех упоминаний - а остальные назовем other

In [ ]:
cuisine_short_set = set()
for item in f_tolist:
    cuisine_short_set.add(item)

In [ ]:
def find_item(cell):
    if item in cell:
        return 1
    return 0
#функция, которая вернет dummies в колонки с типами кухонь

In [ ]:
for item in cuisine_short_set:
    df[item] = df['Cuisine new'].apply(find_item)

In [ ]:
df.shape # к датасету прибавилось 23 колонки

In [ ]:
data = df.copy() #сделаем копию на всякий пожарный

## REVIEWS и выделение дат

In [ ]:
import datetime
from datetime import datetime, timedelta
# здесь я работаю с датами, которые содержатся в колонке Review: я их выделяю и перевожу в формат to_datetime

In [ ]:
data['Reviews'].isna() # есть два пропуска

In [ ]:
data['Reviews'] = data['Reviews'].fillna('no data')

In [ ]:
data['Reviews'].isna().sum()

In [ ]:
data['Reviews'] = data['Reviews'].apply(lambda x: x.split(',')) #сначала я разделяю каждую запись в Review на элементы списка

In [ ]:
len(data['Reviews'][1]) #проверим: теперь каждая запись - это список из нескольких элементов

In [ ]:
data['Reviews'][1] #первые два элемента - это текст отзыва, а вторые два - даты, когда был оставлен отзыв. Они-то нам и нужны!

In [ ]:
def cut_last(line): # эта функция выделит мне последние даты из каждой строки.
    return line[-1][2:-3]

In [ ]:
data['Reviews_lastdate'] = data['Reviews'].apply(cut_last) #применяю функцию к колонке и переношу все выделенные даты в отдельную колонку

In [ ]:
data['Reviews_lastdate'].isna().sum()

In [ ]:
data['Reviews_lastdate'] = data['Reviews_lastdate'].apply(lambda x: None if x == 'data' else x)

In [ ]:
def cut_first(line): # а эта функция выделит мне первые даты из каждой строки.
    if len(line)>=2:
        return line[-2][3:-1]

In [ ]:
data['Reviews_firstdate'] = data['Reviews'].apply(cut_first)

In [ ]:
data['Reviews_firstdate'] = data['Reviews_firstdate'].apply(lambda x: x.lower() if x!= None else None) 
#с этой колонкой больше сложностей: в ней иногда попадается текст, написанный чем попало. поэтому на всякий случай
#я перевожу весь текст в строчные буквы

In [ ]:
def cut_again(line):
    if len(line)<2:
        return None
    elif len(line)>=2:
        if line[0] == "'":
            return line[1:]
    else:
        return line

In [ ]:
data['Reviews_lastdate'] = data['Reviews_lastdate'].apply(cut_again)

In [ ]:
data['Reviews_lastdate'] = pd.to_datetime(data['Reviews_lastdate']) #перевожу данные в первой колонке в формат to_datetime

In [ ]:
def replace(line):
    if 'e' in line:
        return None
    if 'a' in line:
        return None
    if 'r' in line:
        return None
    if 'i' in line:
        return None
    if 'o' in line:
        return None
    if 'u' in line:
        return None
    if 't' in line:
        return None
    if 'w' in line:
        return None
    if '厅' in line:
        return None
    if 'm' in line:
        return None
    if '/' not in line:
        return None
    return line
#ищу текст в колонке и заменяю его на None

In [ ]:
data[data['Reviews_firstdate'].isna()==True]

In [ ]:
data['Reviews_firstdate'] = data['Reviews_firstdate'].fillna('o') 
#пока я none-значения не заменила на букву, у меня не работала функция....

In [ ]:
data['Reviews_firstdate'] = data['Reviews_firstdate'].apply(replace) 
#теперь я могу применитьфункцию по удалению текста

In [ ]:
data['Reviews_firstdate'].isna().sum()

In [ ]:
data['Reviews_firstdate'].isna().sum()
#проверим, работает ли функция  - да, работает, в колонке появились NONE-значения

In [ ]:
data['Reviews_firstdate'] = pd.to_datetime(data['Reviews_firstdate'])
#переведем в to_datetime

In [ ]:
data['Reviews_deltadates'] = data['Reviews_lastdate'] - data['Reviews_firstdate']

In [ ]:
display(data.head())

In [ ]:
print(data['Reviews_deltadates'].describe(datetime_is_numeric=True)) # ну и вызовем статистику
print()
print(data['Reviews_firstdate'].describe(datetime_is_numeric=True))
print()
print(data['Reviews_lastdate'].describe(datetime_is_numeric=True))

In [ ]:
data['ordinal_firstdate']=data['Reviews_firstdate'].map(datetime.toordinal)
#поскольку линейная регрессия не умеет работает с данными типа "даты", переведем все даты в ординальный формат

In [ ]:
data['ordinal_lastdate']=data['Reviews_lastdate'].map(datetime.toordinal)

In [ ]:
data['Reviews_lastdate'][10]

In [ ]:
data['ordinal_timedelta'] = data['ordinal_lastdate'] - data['ordinal_firstdate']
#здесь я создаю новую колонку - разница во времени между последним и первым отзывом. Возможно, это важно.

In [ ]:
data['ordinal_timedelta'] 
# и проверим, как это выглядит. интересно же. Отрицательные числа говорят о том, что
# иногда дата в колонке firstdate оказывается более свежей, чем в колонке lastdate

In [ ]:
latest = data.copy() 
#здесь я хочу создать еще одну колонку, в которой содержатся только самые свежие даты отзывов,
#но специального метода не знаю, поэтому сделаю это как получается :) создаю копию датасета

In [ ]:
latest = latest.loc[latest['ordinal_firstdate'] >= latest['ordinal_lastdate']]
#выделяю в датасете только те строки, где дата из первой колонки- позже даты из второй колонки

In [ ]:
latest

In [ ]:
latest['latest_review_date'] = latest['ordinal_firstdate']
# создаю новую колонку, в которую переношу самые свежие даты из ordinal first_date

In [ ]:
latest2 = data.copy() #дальше повторяю всю ту же процедуру для другой колонки:

In [ ]:
latest2 = latest2.loc[latest2['ordinal_firstdate'] < latest2['ordinal_lastdate']]
#выделяю только те строки, где самые свежие даты содержатся в колонке lastdate

In [ ]:
latest2['latest_review_date'] = latest2['ordinal_lastdate']

In [ ]:
new_df = pd.concat([latest, latest2], ignore_index=True) #объединяю два датасета, так что у меня теперь появляется новая целая колонка

In [ ]:
new_df.info()

In [ ]:
## хотя я и потратила кучу сил на то, чтобы создать эту отдельную колонку с самыми свежими датами ревью, однако она
## она никак не улучшает модель.  Лучшие результаты достигаются с колонкой ordinal_lastdate или ordinal_timedelta

## PRICE RANGE

In [ ]:
new_df['Price Range'].isnull().sum() #посмотрим на количество пропусков

In [ ]:
new_df['Price Range'].value_counts() # а это распределение разных ценовых категорий по датасету

In [ ]:
def price_range(x): # функция, которая меняет категориальное значение числовым. 
    if x == '$':
        return 1
    if x == '$$ - $$$':
        return 2.5
    if x == '$$$$':
        return 4   

In [ ]:
new_df['price_range_index'] = new_df['Price Range'].apply(price_range) #создаю новую колонку с числовыми значениями

In [ ]:
#попробуем заменить NaN в пропорциональном соотношении:
#общая сумма всех заполненных ячеек:
print(f'sum = {23041+7816+1782}')
print(f'1 = {7816/32639}')
print(f'2.5={23041/32639}')
print(f'4 = {1782/32639}')
print('Количество пропусков: ', new_df['price_range_index'].isna().sum())

In [ ]:
#посчитаем, сколько каких значений нам нужно отдать пустым ячейкам:
print(f'1: {round(17359*0.23946)}')
print(f'2.5: {round(17359*0.705934)}')
print(f'4: {round(17359*0.05459)}')

In [ ]:
new_df[new_df['price_range_index'].isna()][0:4157]

In [ ]:
new_df['price_range_index'] = new_df['price_range_index'][0:13235].fillna(1)

In [ ]:
new_df[new_df['price_range_index'].isna()][0:12255]

In [ ]:
new_df['price_range_index'] = new_df['price_range_index'][0:25490].fillna(2.5) #заполним 9790 ячеек значением 2.5

In [ ]:
new_df['price_range_index'] = new_df['price_range_index'].fillna(4) #остальные ячейки заполним значением 4

In [ ]:
new_df['price_range_index'].isna().sum()  # проверяем - все пустые ячейки заполнены

## number of reviews

In [ ]:
new_df['Number of Reviews'].isna().sum() #посмотрим на пропуски в этой колонке

In [ ]:
new_df['Number of Reviews'] = new_df['Number of Reviews'].fillna(0)
#заполним их нулями - раз отзывов нет, значит, скорее всего, их, и правда, нет

In [ ]:
new_df['Number of Reviews'].isna().sum() # проверим, как все заполнилось

In [ ]:
new_df['Number of Reviews'].median() #посмотрим на медианное значение по количеству отзывов

In [ ]:
new_df['Number of Reviews'].describe() # виден очень большой разброс между медианным и средним значением

In [ ]:
new_df['Number of Reviews'].hist(bins=50)

In [ ]:
sns.boxplot(new_df['Number of Reviews']); #посмотрим на выбросы на графике:

In [ ]:
IQR = new_df['Number of Reviews'].quantile(0.75) - new_df['Number of Reviews'].quantile(0.25)
perc25 = new_df['Number of Reviews'].quantile(0.25)
perc75 = new_df['Number of Reviews'].quantile(0.75)
print('25 квантиль -', perc25, '75 квантиль -', perc75)
print(f'верхняя граница выбросов: {perc75+1.5*IQR}')

я попробовала избавиться от выбросов, но это ухудшает модель. Во-первых, их довольно много - около 15 процентов.
во-вторых, большое количество отзывов как раз говорит о том, что это очень популярные рестораны -значит, они качественно значимы для датасета.

## Ranking

In [ ]:
new_df['Ranking'].isna().sum() #посмотрим на пропуски здесь

In [ ]:
new_df['Ranking'].describe()

In [ ]:
new_df['Ranking'].hist();

In [ ]:
sns.boxplot(new_df['Ranking']); #похоже, есть выбросы

In [ ]:
IQR = new_df.Ranking.quantile(0.75) - new_df.Ranking.quantile(0.25)
perc25 = new_df.Ranking.quantile(0.25)
perc75 = new_df.Ranking.quantile(0.75)
print(f'верхняя граница выбросов: {perc75+1.5*IQR}')

In [ ]:
#new_df.Ranking = new_df.Ranking.apply(lambda x: 0 if x >11645 else x)

я попробовала заменить выбросы нулями, но это ухудшило модель.

## Rating

In [ ]:
new_df.Rating.isna().sum()# в целевой переменной выбросов нет

In [ ]:
new_df.Rating.describe() 

In [ ]:
new_df.Rating.hist(bins=20)

In [ ]:
sns.boxplot(new_df.Rating)
#видно, что в основном рестораны получают средние оценки, с крепкой "четверкой" по медиане

## финальное избавление от лишних столбцов:

In [ ]:
new_df.info()
#нужно избавиться от столбцов с категориальными переменными и переменными типа datetime:

In [ ]:
new_df = new_df.drop(['Restaurant_id', 'Cuisine new', 'Reviews_lastdate', 'Reviews_firstdate', 'Reviews_deltadates','Reviews', 'Price Range','Cuisine Style', 'ordinal_firstdate', 'ordinal_timedelta', 'latest_review_date', 'URL_TA', 'ID_TA'], axis=1)

In [ ]:
# Теперь выделим тестовую часть
train_data = new_df.query('sample == 1').drop(['sample'], axis=1)
test_data = new_df.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
#X = train_data.drop(['Rating'], axis=1)
X = chance

In [ ]:
test_data.shape

Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.

In [ ]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
# Загружаем специальный инструмент для разбивки:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

## Submission на Kaggle

In [ ]:
test_data = test_data.drop(['Rating'], axis=1) #возьмем датасет, выделенный специально для тестирования модели, 

In [ ]:
predict_submission = regr.predict(test_data) #применим предсказательную модель к тестовому датасету

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission #перезапишем столбец Rating с учетом новых данных

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('submission_draft11.csv', index=False) # и запишем новый результат в файл